In [ ]:
import pandas as pd
from sqlalchemy import create_engine, exc, text
from sqlalchemy.engine import URL

# Configuration
DB_CONFIG = {
    'host': 'localhost',
    'database': 'strava_poc',
    'user': 'strava_admin',
    'password': 'aqwzsxedc',
    'port': '5432'
}

HR_FILE = 'data/DonneesRH.xlsx'
ACTIVITIES_CSV = 'strava_simulation.csv'

def create_db_connection():
    """Create a database connection"""
    connection_url = URL.create(
        "postgresql+psycopg2",
        username=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        host=DB_CONFIG['host'],
        port=DB_CONFIG['port'],
        database=DB_CONFIG['database']
    )
    return create_engine(connection_url)

In [ ]:


def main():
    print("\nStarting data validation process with corrected ID handling...")
    
    try:
        # Load HR data
        hr_df = pd.read_excel(HR_FILE).rename(columns={
            'ID salarié': 'employee_id',  # HR file has correct employee IDs
            'Nom': 'last_name',
            'Prénom': 'first_name'
        })
        
        # Load Activities data - using ID_salarié as the employee reference
        activities_df = pd.read_csv(ACTIVITIES_CSV).rename(columns={
            'ID_salarié': 'employee_id',  # The true employee ID
            'employee_id': 'activity_id'  # Rename the mislabeled column
        })
        
        # Connect to database
        engine = create_db_connection()
        
        # Verify matching IDs
        hr_ids = set(hr_df['employee_id'])
        activity_employee_ids = set(activities_df['employee_id'])
        missing_ids = activity_employee_ids - hr_ids
        
        if missing_ids:
            print(f"\nVALIDATION ISSUE: {len(missing_ids)} activity records reference non-existent employees")
            print("Sample problematic employee IDs:", sorted(missing_ids)[:5])
            
            # Create diagnostic report
            problem_records = activities_df[activities_df['employee_id'].isin(missing_ids)]
            print("\nSample problematic records:")
            print(problem_records[['employee_id', 'activity_id', 'Type']].head(5))
            
            # Calculate percentage of bad records
            total_activities = len(activities_df)
            bad_percentage = (len(problem_records) / total_activities) * 100
            print(f"\n{len(problem_records)} of {total_activities} records ({bad_percentage:.2f}%) have invalid employee references")
            
            print("\nRECOMMENDED ACTIONS:")
            print("1. Verify if these employee IDs should exist in HR data")
            print("2. If these are test data, filter them out before processing")
            print("3. If these are valid employees, add them to DonneesRH.xlsx")
        else:
            print("\nSUCCESS: All activity records reference valid employees")
            
            # Proceed with database operations
            with engine.connect() as conn:
                # Get existing employee IDs
                existing_ids = pd.read_sql("SELECT employee_id FROM employees", conn)['employee_id'].tolist()
                
                # Find new employees to add
                new_employees = hr_df[~hr_df['employee_id'].isin(existing_ids)]
                
                if not new_employees.empty:
                    print(f"\nInserting {len(new_employees)} new employees...")
                    new_employees.to_sql(
                        'employees', 
                        engine, 
                        if_exists='append', 
                        index=False,
                        method='multi'
                    )
                    print("Employee data successfully updated")
                else:
                    print("\nNo new employees to add")
                
                # Process activities (example)
                print("\nSample of valid activities to process:")
                valid_activities = activities_df[activities_df['employee_id'].isin(hr_ids)]
                print(valid_activities[['employee_id', 'Type', 'Date_de_début']].head(5))
                
    except Exception as e:
        print(f"\nERROR: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    import sys
    main()


Starting data validation process with corrected ID handling...

SUCCESS: All activity records reference valid employees

No new employees to add

Sample of valid activities to process:
   employee_id      Type               Date_de_début
0        59019      Vélo  2024-09-27 11:10:36.283923
1        59019  Escalade  2024-08-13 14:24:08.283923
2        59019    Marche  2025-07-07 06:30:13.283923
3        59019      Yoga  2024-12-20 16:05:41.283923
4        59019      Yoga  2024-11-21 11:47:50.283923


In [16]:
import pandas as pd
from sqlalchemy import create_engine

# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'database': 'strava_poc',
    'user': 'strava_admin',
    'password': 'aqwzsxedc',
    'port': '5432'
}

# Create connection and fetch data
try:
    # Create engine
    engine = create_engine(
        f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
        f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    )
    
    # Query all employees
    employees_df = pd.read_sql("SELECT * FROM employees", engine)
    
    # Display the DataFrame
    print("Employee Data from PostgreSQL:")
    print(employees_df)
    
    # Optional: Show DataFrame info
    print("\nDataFrame Info:")
    employees_df.info()

except Exception as e:
    print(f"Error accessing database: {e}")

Employee Data from PostgreSQL:
     employee_id first_name  last_name  \
0          59019     Audrey      Colin   
1          19841    Monique     Ledoux   
2          56482   Michelle     Dumont   
3          21886     Judith  Toussaint   
4          81001   Michelle     Bailly   
..           ...        ...        ...   
156        18941     Gilles    Guillou   
157        81676   Jeannine     Breton   
158        27069   Philippe   Delahaye   
159        30256     Odette      Dumas   
160        94680      Henri     Pineau   

                                         home_address distance_to_work  \
0                   128 Rue du Port, 34000 Frontignan             None   
1      68 Rue du Port, 34970 Saint-Clément-de-Rivière             None   
2                     100 Av. de la Gare, 30900 Nîmes             None   
3                     53 Av. de la Gare, 34970 Lattes             None   
4                     74 Rue des Fleurs, 34970 Lattes             None   
..                  

In [20]:
import pandas as pd

def prepare_strava_data(input_path, output_path):
    # Read the CSV with proper encoding
    df = pd.read_csv(input_path, encoding='utf-8-sig')
    
    # Standardize column names
    column_map = {
        'ID_salarié': 'employee_id',
        'Date_de_début': 'start_date',
        'Type': 'sport_type',
        'Distance': 'distance_meters',
        'Date_de_fin': 'end_date',
        'Commentaire': 'comment'
    }
    df = df.rename(columns={k: v for k, v in column_map.items() if k in df.columns})
    
    # Calculate elapsed time if both dates exist
    if 'start_date' in df.columns and 'end_date' in df.columns:
        df['elapsed_time_seconds'] = (
            pd.to_datetime(df['end_date']) - 
            pd.to_datetime(df['start_date'])
        ).dt.total_seconds().astype(int)
    
    # Select only the columns we want to import (exclude id and is_valid)
    final_columns = [
        'employee_id',
        'start_date',
        'sport_type',
        'distance_meters',
        'elapsed_time_seconds',
        'end_date',
        'comment'
    ]
    df = df[[col for col in final_columns if col in df.columns]]
    
    # Save with proper header and no index
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"File successfully prepared at: {output_path}")

# Usage
prepare_strava_data(
    input_path='C:/temp/strava_simulation.csv',
    output_path='C:/temp/strava_ready_to_import.csv'
)

File successfully prepared at: C:/temp/strava_ready_to_import.csv
